In [ ]:
from IPython.display import clear_output
from pyomo.environ import (
    ConcreteModel,
    Set,
    value,
    Constraint,
    Expression,
    Var,
    log10,
    exp,
    TransformationFactory,
    assert_optimal_termination,
    units as pyunits,
)
from pyomo.network import Arc

from idaes.models.unit_models import (
    Separator,
    Mixer,
    StateJunction,
    Feed,
    Product,
    MomentumMixingType,
    SplittingType,
)
from idaes.core import (
    FlowsheetBlock,
    UnitModelCostingBlock,
    MaterialBalanceType,
    EnergyBalanceType,
    MomentumBalanceType,
)
from idaes.core.util.initialization import propagate_state
from idaes.core.util.model_statistics import degrees_of_freedom
import idaes.core.util.scaling as iscale

from watertap.unit_models.uv_aop import Ultraviolet0D, UVDoseType
from watertap.property_models.unit_specific.dioxane_prop_pack import (
    DioxaneParameterBlock,
)
from watertap.property_models.unit_specific.NDMA_prop_pack import NDMAParameterBlock
from watertap.core.solvers import get_solver
from watertap.costing import WaterTAPCosting
from watertap.core.util.model_diagnostics.infeasible import *

solver = get_solver()

In [ ]:
def touch_flow_and_conc(b):
    """
    Touch flow and conc variables for construction
    """
    props = b.find_component("properties")
    if props is not None:
        props[0].flow_vol_phase
        props[0].conc_mass_phase_comp
        return

    cv = b.find_component("control_volume")
    if cv is not None:
        cv.properties_in[0].flow_vol_phase
        cv.properties_in[0].conc_mass_phase_comp
        cv.properties_out[0].flow_vol_phase
        cv.properties_out[0].conc_mass_phase_comp
        return

    b.mixed_state[0].flow_vol_phase
    b.mixed_state[0].conc_mass_phase_comp

    if isinstance(b, Mixer):
        for x in b.config.inlet_list:
            sb = b.find_component(f"{x}_state")
            sb[0].flow_vol_phase
            sb[0].conc_mass_phase_comp
    if isinstance(b, Separator):
        for x in b.config.outlet_list:
            sb = b.find_component(f"{x}_state")
            sb[0].flow_vol_phase
            sb[0].conc_mass_phase_comp

In [ ]:
def report_feed(blk, funits=None, cunits=None, w=25):

    if funits is None:
        funits = pyunits.m**3 / pyunits.day
    if cunits is None:
        cunits = pyunits.ng / pyunits.L

    # fs = blk.flowsheet()
    title = blk.name.replace("fs.", "").replace("_", " ").upper()

    side = int(((3 * w) - len(title)) / 2) - 1
    header = "=" * side + f" {title} " + "=" * side
    print(f"\n{header}\n")
    ms = blk.find_component("properties")
    flow_in = value(
        pyunits.convert(
            ms[0].flow_vol_phase["Liq"],
            to_units=funits,
        )
    )
    conc_in = value(
        pyunits.convert(
            ms[0].conc_mass_phase_comp["Liq", "NDMA"],
            to_units=cunits,
        )
    )
    print(f'{"FEED Flow":<{w}s}{f"{flow_in:<{w},.1f}"}{f"{funits}":<{w}s}')
    print(f'{"FEED NDMA":<{w}s}{f"{conc_in:<{w},.1f}"}{f"{cunits}":<{w}s}')


def report_separator(blk, funits=None, cunits=None, w=25):

    if funits is None:
        funits = pyunits.m**3 / pyunits.day
    if cunits is None:
        cunits = pyunits.ng / pyunits.L

    # fs = blk.flowsheet()
    title = blk.name.replace("fs.", "").replace("_", " ").upper()

    side = int(((3 * w) - len(title)) / 2) - 1
    header = "=" * side + f" {title} " + "=" * side
    print(f"\n{header}\n")
    ms = blk.find_component("mixed_state")
    flow_in = value(
        pyunits.convert(
            ms[0].flow_vol_phase["Liq"],
            to_units=funits,
        )
    )
    conc_in = value(
        pyunits.convert(
            ms[0].conc_mass_phase_comp["Liq", "NDMA"],
            to_units=cunits,
        )
    )
    print(f'{"INLET Flow":<{w}s}{f"{flow_in:<{w},.1f}"}{f"{funits}":<{w}s}')
    print(f'{"INLET NDMA":<{w}s}{f"{conc_in:<{w},.1f}"}{f"{cunits}":<{w}s}')
    tot_flow_out = sum(
        value(
            value(
                pyunits.convert(
                    blk.find_component(f"{x}_state")[0].flow_vol_phase["Liq"],
                    to_units=funits,
                )
            )
        )
        for x in blk.config.outlet_list
    )
    print(f'{"TOTAL OUTLET FLOW":<{w}s}{f"{tot_flow_out:<{w},.1f}"}{f"{funits}":<{w}s}')
    for x in blk.config.outlet_list:
        sb = blk.find_component(f"{x}_state")
        flow_out = value(
            pyunits.convert(
                sb[0].flow_vol_phase["Liq"],
                to_units=funits,
            )
        )
        conc_out = value(
            pyunits.convert(
                sb[0].conc_mass_phase_comp["Liq", "NDMA"],
                to_units=cunits,
            )
        )
        print(
            f'{"   Flow " + x.replace("_", " ").title():<{w}s}{f"{flow_out:<{w},.1f}"}{f"{funits}":<{w}s}'
        )
        print(
            f'{"   NDMA " + x.replace("_", " ").title():<{w}s}{f"{conc_out:<{w},.1f}"}{f"{cunits}":<{w}s}'
        )


def report_mixer(mixer, funits=None, cunits=None, w=25):

    if funits is None:
        funits = pyunits.m**3 / pyunits.day
    if cunits is None:
        cunits = pyunits.ng / pyunits.L
    ms = mixer.find_component("mixed_state")
    title = mixer.name.replace("fs.", "").replace("_", " ").upper()
    side = int(((3 * w) - len(title)) / 2) - 1
    header = "=" * side + f" {title} " + "=" * side
    print(f"\n{header}\n")
    tot_flow_in = sum(
        value(
            pyunits.convert(
                mixer.find_component(f"{x}_state")[0].flow_vol_phase["Liq"],
                to_units=funits,
            )
        )
        for x in mixer.config.inlet_list
    )
    print(f'{"TOTAL INLET FLOW":<{w}s}{f"{tot_flow_in:<{w},.1f}"}{f"{funits}":<{w}s}')
    for x in mixer.config.inlet_list:
        sb = mixer.find_component(f"{x}_state")
        flow_in = value(
            pyunits.convert(
                sb[0].flow_vol_phase["Liq"],
                to_units=funits,
            )
        )
        conc_in = value(
            pyunits.convert(
                sb[0].conc_mass_phase_comp["Liq", "NDMA"],
                to_units=cunits,
            )
        )
        p_in = value(
            pyunits.convert(
                sb[0].pressure,
                to_units=pyunits.psi,
            )
        )
        print(
            f'{"   Flow " + x.replace("_", " ").title():<{w}s}{f"{flow_in:<{w},.1f}"}{f"{funits}":<{w}s}'
        )
        print(
            f'{"   NDMA " + x.replace("_", " ").title():<{w}s}{f"{conc_in:<{w},.1f}"}{f"{cunits}":<{w}s}'
        )
        print(
            f'{"   Pressure " + x.replace("_", " ").title():<{w}s}{f"{p_in:<{w},.1f}"}{"psi":<{w}s}'
        )
    flow_out = value(
        pyunits.convert(
            ms[0].flow_vol_phase["Liq"],
            to_units=funits,
        )
    )
    conc_out = value(
        pyunits.convert(
            ms[0].conc_mass_phase_comp["Liq", "NDMA"],
            to_units=cunits,
        )
    )
    p_out = value(
        pyunits.convert(
            ms[0].pressure,
            to_units=pyunits.psi,
        )
    )
    print(f'{"Outlet Flow":<{w}s}{f"{flow_out:<{w},.1f}"}{f"{funits}":<{w}s}')
    print(f'{"Outlet NDMA":<{w}s}{f"{conc_out:<{w},.1f}"}{f"{cunits}":<{w}s}')
    print(f'{"Outlet Pressure":<{w}s}{f"{p_out:<{w},.1f}"}{"psi":<{w}s}')

In [ ]:
def report_uv(blk, funits=None, cunits=None, w=40):
    comps = [
        "inactivation_rate",
        "rate_constant",
        "photolysis_rate_constant",
        "reaction_rate_constant",
        "uv_dose",
        "uv_intensity",
        "exposure_time",
        "reactor_volume",
        "electricity_demand_phase_comp",
        "max_phase_electricity_demand",
        "electricity_demand_comp",
        "max_component_electricity_demand",
        "electricity_demand",
        "electrical_efficiency_phase_comp",
        "lamp_efficiency",
    ]

    if blk.config.has_aop:
        comps.extend(
            [
                "hydrogen_peroxide_dose",
                "hydrogen_peroxide_conc",
                "second_order_reaction_rate_constant",
                "reaction_rate_constant",
            ]
        )

    if funits is None:
        funits = pyunits.m**3 / pyunits.day
    if cunits is None:
        cunits = pyunits.ng / pyunits.L

    title = blk.name.replace("fs.", "").replace("_", " ").upper()

    side = int(((3 * w) - len(title)) / 2) - 1
    header = "=" * side + f" {title} " + "=" * side
    print(f"\n{header}\n")
    cv = blk.find_component("control_volume")
    flow_in = value(
        pyunits.convert(
            cv.properties_in[0].flow_vol_phase["Liq"],
            to_units=funits,
        )
    )
    conc_in = value(
        pyunits.convert(
            cv.properties_in[0].conc_mass_phase_comp["Liq", "NDMA"],
            to_units=cunits,
        )
    )
    print(f'{"INLET Flow":<{w}s}{f"{flow_in:<{w},.1f}"}{f"{funits}":<{w}s}')
    print(f'{"INLET NDMA":<{w}s}{f"{conc_in:<{w},.1f}"}{f"{cunits}":<{w}s}')
    flow_out = value(
        pyunits.convert(
            cv.properties_out[0].flow_vol_phase["Liq"],
            to_units=funits,
        )
    )
    conc_out = value(
        pyunits.convert(
            cv.properties_out[0].conc_mass_phase_comp["Liq", "NDMA"],
            to_units=cunits,
        )
    )
    print(f'{"OUTLET Flow":<{w}s}{f"{flow_out:<{w},.1f}"}{f"{funits}":<{w}s}')
    print(f'{"OUTLET NDMA":<{w}s}{f"{conc_out:<{w},.1f}"}{f"{cunits}":<{w}s}')
    for c in comps:
        v = blk.find_component(c)
        if v is not None:
            if v.is_indexed():
                for i, vv in v.items():
                    val = value(vv)
                    print(
                        f'{c.replace("_", " ").title():<{w}s}{f"{val:<{w},.4f}"}{f"{pyunits.get_units(vv)}":<{w}s}'
                    )
            else:
                val = value(v)
                print(
                    f'{c.replace("_", " ").title():<{w}s}{f"{val:<{w},.4f}"}{f"{pyunits.get_units(v)}":<{w}s}'
                )


def report_costing(m, w=40):

    title = "COSTING REPORT"
    side = int(((3 * w) - len(title)) / 2) - 1
    header = "=" * side + f" {title} " + "=" * side
    print(f"\n{header}\n")
    for c in [
        "total_capital_cost",
        "total_operating_cost",
        "LCOW",
        "SEC",
        "aggregate_flow_electricity",
        "aggregate_flow_h2o2",
        "aggregate_flow_costs",
    ]:
        v = m.fs.costing.find_component(c)

        if v is not None:
            if v.is_indexed():
                for i, vv in v.items():
                    val = value(vv)
                    print(
                        f'{f"Aggregate Flow {i.upper()}":<{w}s}{f"{val:<{w},.4f}"}{f"{pyunits.get_units(vv)}":<{w}s}'
                    )
            else:
                val = value(v)
                print(
                    f'{c.replace("_", " ").title():<{w}s}{f"{val:<{w},.4f}"}{f"{pyunits.get_units(v)}":<{w}s}'
                )

In [ ]:
def report_system(m, funits=None, cunits=None, w=40):
    if funits is None:
        funits = pyunits.m**3 / pyunits.day
    if cunits is None:
        cunits = pyunits.ng / pyunits.L

    report_feed(m.fs.feed, funits=funits, cunits=cunits, w=w)
    report_separator(m.fs.separator, funits=funits, cunits=cunits, w=w)
    for r in m.fs.reactor_set:
        uv = m.fs.find_component(r)
        report_uv(uv, funits=funits, cunits=cunits, w=w)
    report_mixer(m.fs.mixer, funits=funits, cunits=cunits, w=w)
    report_feed(m.fs.product, funits=funits, cunits=cunits, w=w)

In [ ]:
##################################################
# Conditions
##################################################

rho = 1000 * pyunits.kg / pyunits.m**3  # [kg/m3], density of water
flow_rate_vol = (
    163 * pyunits.m**3 / pyunits.day
)  # [m3/day], based capacity of LPUV lamp used in Project 1550
influent_conc = (
    0.041 * pyunits.ug / pyunits.liter
)  # concentration of NDMA [ug/L] entering UV AOP reactor
# Demo conditions for
# lamp_type = 'LPUV'
irradiated_surf_area = (
    0.103 * pyunits.m**2
)  # [m2], this must be the units to have electricity_demand in W
reactor_vol = 0.0081 * pyunits.m**3  # [m3], volume of UV reactor used in Project 1550
h2o2_dose = 1 * pyunits.mg / pyunits.liter
flow_rate_mass_water = flow_rate_vol * rho  # [kg/day], mass flow rate of water
flow_rate_mass_comp = flow_rate_vol * influent_conc

num_units = 5
even_split = 1 / num_units
reactor_list = [f"uv{i+1}" for i in range(num_units)]
outlet_list = [f"to_uv{i+1}" for i in range(num_units)]
inlet_list = [f"from_uv{i+1}" for i in range(num_units)]

flow_per_reactor = flow_rate_vol / num_units

##################################################
# Build model
##################################################

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NDMAParameterBlock()
m.fs.costing = WaterTAPCosting()
m.fs.reactor_set = Set(initialize=reactor_list)

m.fs.feed = Feed(property_package=m.fs.properties)
touch_flow_and_conc(m.fs.feed)

m.fs.separator = Separator(
    property_package=m.fs.properties,
    split_basis=SplittingType.totalFlow,
    outlet_list=outlet_list,
)
touch_flow_and_conc(m.fs.separator)

for r in m.fs.reactor_set:
    m.fs.add_component(
        r,
        Ultraviolet0D(
            dynamic=False,
            has_holdup=False,
            material_balance_type=MaterialBalanceType.useDefault,
            energy_balance_type=EnergyBalanceType.useDefault,
            momentum_balance_type=MomentumBalanceType.pressureTotal,
            property_package=m.fs.properties,
            has_pressure_change=False,
            uv_dose_type=UVDoseType.fixed,
            has_aop=True,
            target_species=None,
        ),
    )
    uv = m.fs.find_component(r)
    touch_flow_and_conc(uv)
    uv.log_removal = Expression(
        expr=log10(
            uv.control_volume.properties_in[0].conc_mass_phase_comp["Liq", "NDMA"]
            / uv.control_volume.properties_out[0].conc_mass_phase_comp["Liq", "NDMA"]
        )
    )
    uv.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)

m.fs.mixer = Mixer(
    property_package=m.fs.properties,
    momentum_mixing_type=MomentumMixingType.minimize,
    inlet_list=inlet_list,
)
touch_flow_and_conc(m.fs.mixer)

m.fs.product = Product(property_package=m.fs.properties)
touch_flow_and_conc(m.fs.product)


m.fs.costing.cost_process()
m.fs.costing.add_LCOW(flow_rate=m.fs.product.properties[0].flow_vol_phase["Liq"])
m.fs.costing.add_specific_energy_consumption(
    flow_rate=m.fs.product.properties[0].flow_vol_phase["Liq"], name="SEC"
)

###################################################
# Arcs
##################################################

m.fs.feed_to_separator = Arc(source=m.fs.feed.outlet, destination=m.fs.separator.inlet)

for r in m.fs.reactor_set:
    uv = m.fs.find_component(r)
    s = m.fs.separator.find_component(f"to_{r}")
    a = Arc(source=s, destination=uv.inlet)
    m.fs.add_component(f"sep_to_{r}", a)

    d = m.fs.mixer.find_component(f"from_{r}")
    a = Arc(source=uv.outlet, destination=d)
    m.fs.add_component(f"{r}_to_mix", a)

m.fs.mixer_to_product = Arc(source=m.fs.mixer.outlet, destination=m.fs.product.inlet)

TransformationFactory("network.expand_arcs").apply_to(m)

##################################################
# Set Operating Conditions
##################################################

m.fs.costing.base_currency = pyunits.USD_2023
m.fs.costing.ultraviolet.factor_lamp_replacement.fix(0.73)
m.fs.costing.ultraviolet.reactor_cost.fix(203.4)
m.fs.costing.ultraviolet.lamp_cost.fix(8289)

for i, r in enumerate(m.fs.reactor_set, 1):
    uv = m.fs.find_component(r)
    uv.lamp_efficiency.fix(0.375)
    # overall inactivation rate constant for NDMA [m2/J]; degredation is primarily driven by direct photolysis for NDMA
    uv.inactivation_rate.fix(2.29e-3)
    uv.uv_dose.fix(22320)
    # [m3], based on volume of UV reactor used in Project 1550
    uv.reactor_volume.fix(reactor_vol)
    exposure_time = pyunits.convert(reactor_vol / flow_per_reactor, to_units=pyunits.s)
    uv.exposure_time.fix(exposure_time)

    elec_demand = pyunits.convert(
        (uv.uv_dose * irradiated_surf_area) / (uv.lamp_efficiency * uv.exposure_time),
        to_units=pyunits.W,
    )
    uv.electricity_demand.fix(elec_demand)

    intensity = pyunits.convert(
        uv.uv_dose / uv.exposure_time, to_units=pyunits.W / pyunits.m**2
    )
    uv.uv_intensity.fix(intensity)
    uv.hydrogen_peroxide_dose.fix(h2o2_dose)
    uv.outlet.pressure.fix(101325)

    print(f"dof for {r} = {degrees_of_freedom(uv)}")

    if i != 1:
        # need to leave one split fraction unfixed to avoid overconstraining
        m.fs.separator.split_fraction[0, f"to_{r}"].fix(even_split)


##################################################
# Scaling
##################################################

m.fs.properties.set_default_scaling(
    "flow_mass_phase_comp",
    1 / value(pyunits.convert(flow_rate_mass_water, to_units=pyunits.kg / pyunits.s)),
    index=("Liq", "H2O"),
)
m.fs.properties.set_default_scaling(
    "flow_mass_phase_comp",
    1 / value(pyunits.convert(flow_rate_mass_comp, to_units=pyunits.kg / pyunits.s)),
    index=("Liq", "NDMA"),
)

iscale.calculate_scaling_factors(m)


##################################################
# Set inlet conditions
##################################################

m.fs.feed.properties.calculate_state(
    var_args={
        ("flow_vol_phase", ("Liq")): flow_rate_vol,
        ("conc_mass_phase_comp", ("Liq", "NDMA")): influent_conc,
        ("pressure", None): 101325,
        ("temperature", None): 273.15 + 25,
    },
    hold_state=True,
)
##################################################
# Initialize and propagate
##################################################

propagate_state(m.fs.feed_to_separator)

m.fs.separator.initialize()

for r in m.fs.reactor_set:
    uv = m.fs.find_component(r)
    a = m.fs.find_component(f"sep_to_{r}")
    propagate_state(a)

    uv.initialize()

    a = m.fs.find_component(f"{r}_to_mix")
    propagate_state(a)

m.fs.mixer.initialize()
propagate_state(m.fs.mixer_to_product)
m.fs.product.initialize()

##################################################
# Solve
##################################################

print(f"dof = {degrees_of_freedom(m)}")
assert degrees_of_freedom(m) == 0

results = solver.solve(m, tee=False)
assert_optimal_termination(results)

report_system(m)
report_costing(m)